<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/3-introduction-to-keras-and-tensorflow/01_tensorflow_fundametal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##First steps with TensorFlow

As we know, training a neural network revolves around the following
concepts:

- First, low-level tensor manipulation—the infrastructure that underlies all modern machine learning. This translates to TensorFlow APIs:
  - Tensors, including special tensors that store the network’s state (variables)
  - Tensor operations such as addition, relu, matmul
  - Backpropagation, a way to compute the gradient of mathematical expressions

- Second, high-level deep learning concepts. This translates to Keras APIs:
  - Layers, which are combined into a model
  - A loss function, which defines the feedback signal used for learning
  - An optimizer, which determines how learning proceeds
  - Metrics to evaluate model performance, such as accuracy
  - A training loop that performs mini-batch stochastic gradient descent

Now let’s take a deeper dive into how all of these different concepts can be
approached in practice using TensorFlow and Keras.

##Setup

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

##Constant tensors and variables

To do anything in TensorFlow, we’re going to need some tensors. Tensors need to be created with some initial value. 

For instance, you could create all-ones or all-zeros tensors, or tensors of values drawn from a random distribution.

**All-ones or all-zeros tensors**

In [2]:
x = tf.ones(shape=(2, 1))
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)


In [3]:
x = np.ones(shape=(2, 1))
print(x)

[[1.]
 [1.]]


In [4]:
x = tf.zeros(shape=(2, 1))
print(x)

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


In [5]:
x = np.zeros(shape=(2, 1))
print(x)

[[0.]
 [0.]]


**Random tensors**

In [6]:
# Tensor of random values drawn from a normal distribution with mean 0 and standard deviation 1.
x = tf.random.normal(shape=(3, 1), mean=0.0, stddev=1.0)
print(x)

tf.Tensor(
[[-0.17470434]
 [-2.6843026 ]
 [-0.04324743]], shape=(3, 1), dtype=float32)


In [7]:
x = np.random.normal(size=(3, 1), loc=0.0, scale=1.0)
print(x)

[[-1.45274625]
 [-0.35813236]
 [ 0.49590577]]


In [8]:
# Tensor of random values drawn from a uniform distribution between 0 and 1.
x = tf.random.uniform(shape=(3, 1), minval=0.0, maxval=1.0)
print(x)

tf.Tensor(
[[0.0677222 ]
 [0.89348197]
 [0.84030616]], shape=(3, 1), dtype=float32)


In [9]:
x = np.random.uniform(size=(3, 1), low=0.0, high=1.0)
print(x)

[[0.92575194]
 [0.8329919 ]
 [0.90909402]]


A significant difference between NumPy arrays and TensorFlow tensors is that Tensor-
Flow tensors aren’t assignable: they’re constant. 

For instance, in NumPy, you can do
the following.

In [10]:
x = np.ones(shape=(2, 2))
print(x)

[[1. 1.]
 [1. 1.]]


In [12]:
x[0, 0] = 0.0
print(x)

[[0. 1.]
 [1. 1.]]


Try to do the same thing in TensorFlow, and you will get an error: “EagerTensor object
does not support item assignment.”

In [13]:
x = tf.ones(shape=(2, 2))
print(x)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)


In [ ]:
# x[0, 0] = 0.0

```log
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-b9cc6021f76a> in <module>()
----> 1 x[0, 0] = 0.0
      2 print(x)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
```

To train a model, we’ll need to update its state, which is a set of tensors. If tensors aren’t assignable, how do we do it?

That’s where variables come in. `tf.Variable` is the
class meant to manage modifiable state in TensorFlow.

To create a variable, you need to provide some initial value, such as a random tensor.

In [15]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.48890153],
       [-1.4609447 ],
       [-1.5606052 ]], dtype=float32)>


The state of a variable can be modified via its assign method, as follows.

In [16]:
v.assign(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>

It also works for a subset of the coefficients.

In [17]:
v[0, 0].assign((3.0))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

In [20]:
v[1, 0].assign((4.0))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [4.],
       [1.]], dtype=float32)>

Similarly, assign_add() and assign_sub() are efficient equivalents of += and -=, as shown next.

In [21]:
v.assign_add(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [5.],
       [2.]], dtype=float32)>

In [22]:
v.assign_sub(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [4.],
       [1.]], dtype=float32)>

##Tensor operations: Doing math in TensorFlow